In [1]:
import numpy as np
import pandas as pd
from sort.sort import Sort
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
mot_tracker = Sort()

In [2]:
bb_x_width = 10
bb_y_width = 10

def transform(df):
    df['x'] = ( 0.0075 + df["x"] ) / 2.9296875e-05
    df['y'] = ( -0.0225 + df["y"] ) / 2.9296875e-05
    # df['x'] = 256 + (df["x"] * 1000)*512/16 - 4
    # df['y'] = 256 - (30- df["y"]*1000)*512/16 -4
    return df
def createBB(data):
    data['x1'] = data['x'] - bb_x_width/2
    data['y1'] = data['y'] - bb_y_width/2
    data['x2'] = data['x'] + bb_x_width/2
    data['y2'] = data['y'] + bb_y_width/2
    data['score'] = 1.0
    return data
def get_detections(df):
    coords = []
    for row in df.iterrows():
        coords.append([row[1]['x1'], row[1]['y1'], row[1]['x2'], row[1]['y2'], row[1]['score']])
    return np.array(coords)
def get_detections_per_frame(data, frame_no=1):
    df = data[data['frame'] == frame_no]
    detections = get_detections(df)
    return detections

In [3]:
data = pd.read_csv("../data/detected_centLocs.csv")
# data = pd.read_csv("../data/ground_truth.txt", header=None)
# data.columns = ['frame', 'id', 'x', 'z', 'y']
# print(data.head())
data = transform(data)
# print(data.head())
data = createBB(data)
# print(data.head())

In [4]:
# Create figure and axes
fig, ax = plt.subplots()

# Display the image

for frame_no in range(20):
    gt_at_frame = data[data['frame']==frame_no]
    
    gt_bb_at_frame = get_detections_per_frame(data, frame_no)
    track_bbs_ids = mot_tracker.update(gt_bb_at_frame)
    # print(track_bbs_ids)

    # Create a Rectangle patch
    for i in range(len(track_bbs_ids)):
        anchor_x = track_bbs_ids[i][0]
        anchor_y = track_bbs_ids[i][1]
        rect = patches.Rectangle((anchor_x, anchor_y), bb_x_width, bb_y_width, linewidth=1, edgecolor='r', facecolor='none')
        if track_bbs_ids[i][-1] == 50:
            ax.add_patch(rect)
        ax.plot(gt_at_frame.iloc[i]['x'],gt_at_frame.iloc[i]['y'], color = 'b', marker = '+')


# Add the patch to the Axes
im = Image.open(f'../data/frames/frame{frame_no}.jpg')
ax.imshow(im)
plt.show()

In [5]:
# for i in range(len(data_at_frame)):
#     anchor_x = data_at_frame.iloc[i]['x'] - bb_x_width/2
#     anchor_y = data_at_frame.iloc[i]['y'] - bb_y_width/2
# det_per_frame